# Fake News Detection: Data Preprocessing

This notebook demonstrates the preprocessing steps for the fake news detection project, specifically tailored to the characteristics of the True.csv and Fake.csv datasets.

## Overview

In this notebook, we will:
1. Load the raw data from Hive tables
2. Analyze dataset characteristics to inform preprocessing decisions
3. Demonstrate why the 'subject' column must be dropped (data leakage)
4. Apply dataset-specific preprocessing operations
5. Save the preprocessed data for feature engineering

## Setup

First, let's set up our Spark session and import necessary libraries.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, isnan, isnull, desc, expr, lit, trim
import matplotlib.pyplot as plt
import numpy as np
import re
import sys

# Add the project root to the Python path
sys.path.append('/dbfs/FileStore/tables')
sys.path.append('/home/ubuntu/BDA')

# Import custom modules
from BDA.01_data_ingestion.hive_data_ingestion import HiveDataIngestion
from BDA.02_preprocessing.enhanced_text_preprocessor import EnhancedTextPreprocessor, analyze_dataset_characteristics
from BDA.02_preprocessing.date_utils import DateValidator
from BDA.02_preprocessing.data_validation_utils import DataValidator

# Configure Spark session optimized for Databricks Community Edition
spark = SparkSession.builder \
    .appName("FakeNewsDetection_Preprocessing") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.driver.memory", "8g") \
    .enableHiveSupport() \
    .getOrCreate()

# Display Spark configuration
print(f"Spark version: {spark.version}")
print(f"Shuffle partitions: {spark.conf.get('spark.sql.shuffle.partitions')}")
print(f"Driver memory: {spark.conf.get('spark.driver.memory')}")

## Load Data

We'll load the data from the Hive tables ('fake' and 'real').

In [ ]:
# Create an instance of HiveDataIngestion
data_ingestion = HiveDataIngestion(spark)

# Load data from Hive tables
real_df, fake_df = data_ingestion.load_data_from_hive()

# Display sample data
print("Real news sample:")
display(real_df.limit(3))

print("\nFake news sample:")
display(fake_df.limit(3))

## Data Exploration and Validation

Let's explore the datasets to understand their characteristics and identify preprocessing needs.

In [ ]:
# Create a DataValidator instance
validator = DataValidator()

# Check for missing values in both datasets
print("Missing values in real news dataset:")
validator.check_missing_values(real_df).show()

print("\nMissing values in fake news dataset:")
validator.check_missing_values(fake_df).show()

In [ ]:
# Check for duplicates
print(f"Duplicates in real news dataset: {validator.check_duplicates(real_df)}")
print(f"Duplicates in fake news dataset: {validator.check_duplicates(fake_df)}")

In [ ]:
# Check text length distribution
real_text_lengths = real_df.select(expr("size(split(text, ' '))").alias("length"))
fake_text_lengths = fake_df.select(expr("size(split(text, ' '))").alias("length"))

# Convert to pandas for visualization
real_lengths_pd = real_text_lengths.toPandas()
fake_lengths_pd = fake_text_lengths.toPandas()

# Plot text length distributions
plt.figure(figsize=(12, 6))
plt.hist(real_lengths_pd['length'], bins=50, alpha=0.5, label='Real News')
plt.hist(fake_lengths_pd['length'], bins=50, alpha=0.5, label='Fake News')
plt.xlabel('Text Length (words)')
plt.ylabel('Frequency')
plt.title('Text Length Distribution')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Print summary statistics
print("Real news text length statistics:")
real_text_lengths.summary().show()

print("\nFake news text length statistics:")
fake_text_lengths.summary().show()

## Analyzing the 'subject' Column - Data Leakage Issue

Let's examine the 'subject' column in both datasets to understand why it must be dropped.

In [ ]:
# Check subject distribution in real news
print("Subject distribution in real news:")
real_subjects = real_df.groupBy("subject").count().orderBy(desc("count"))
display(real_subjects)

# Check subject distribution in fake news
print("\nSubject distribution in fake news:")
fake_subjects = fake_df.groupBy("subject").count().orderBy(desc("count"))
display(fake_subjects)

### Data Leakage Visualization

The analysis above reveals a critical issue: the 'subject' column perfectly discriminates between real and fake news. This creates a data leakage problem that would invalidate our model.

In [ ]:
# Create a combined dataset with labels
real_with_label = real_df.withColumn("label", lit(1))
fake_with_label = fake_df.withColumn("label", lit(0))
combined_df = real_with_label.union(fake_with_label)

# Get subject distribution by label
subject_by_label = combined_df.groupBy("subject", "label").count().orderBy("subject", "label")
display(subject_by_label)

# Convert to pandas for visualization
subject_label_pd = subject_by_label.toPandas()

# Create a pivot table
pivot_data = subject_label_pd.pivot(index='subject', columns='label', values='count').fillna(0)
pivot_data.columns = ['Fake News', 'Real News']

# Plot the distribution
ax = pivot_data.plot(kind='bar', figsize=(10, 6), color=['#FF6B6B', '#4ECDC4'])
plt.title('Subject Distribution by News Type')
plt.xlabel('Subject')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)

# Add text labels
for container in ax.containers:
    ax.bar_label(container, fmt='%d')

plt.tight_layout()
plt.show()

### Why We Must Drop the 'subject' Column

As demonstrated above, the 'subject' column creates a perfect data leakage problem:

1. **Perfect Correlation with Labels**: 
   - Real news articles are all labeled as "politicsNews"
   - Fake news articles are all labeled as "News"

2. **Model Impact**: If we include this column:
   - The model would achieve nearly 100% accuracy in training and validation
   - But this accuracy would be misleading and wouldn't generalize to real-world data
   - The model would simply learn "if subject='politicsNews' then real, else fake"

3. **Real-World Failure**: In production, new articles wouldn't follow this artificial pattern, causing the model to make incorrect predictions based on an irrelevant feature.

4. **Scientific Integrity**: Including this column would invalidate any research findings since the model wouldn't be learning meaningful patterns in the text content.

Therefore, we must drop the 'subject' column from our preprocessing pipeline to ensure our model learns from the actual content of the news articles rather than this artificial distinction.

## Analyzing URL Presence

Let's check for the presence of URLs in both datasets to confirm our preprocessing needs.

In [ ]:
# Define a function to check for URLs
def contains_url(text):
    if text is None:
        return False
    url_pattern = r'https?://t\.co/\w+|http\S+|www\S+|https\S+'
    return bool(re.search(url_pattern, text))

# Register the function as a UDF
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType
contains_url_udf = udf(contains_url, BooleanType())

# Check URL presence in real news
real_with_url = real_df.withColumn("contains_url", contains_url_udf(col("text")))
real_url_count = real_with_url.filter(col("contains_url") == True).count()
real_total = real_df.count()
real_url_percentage = (real_url_count / real_total) * 100

# Check URL presence in fake news
fake_with_url = fake_df.withColumn("contains_url", contains_url_udf(col("text")))
fake_url_count = fake_with_url.filter(col("contains_url") == True).count()
fake_total = fake_df.count()
fake_url_percentage = (fake_url_count / fake_total) * 100

# Print results
print(f"Real news articles containing URLs: {real_url_count} out of {real_total} ({real_url_percentage:.2f}%)")
print(f"Fake news articles containing URLs: {fake_url_count} out of {fake_total} ({fake_url_percentage:.2f}%)")

# Plot the results
labels = ['Real News', 'Fake News']
url_percentages = [real_url_percentage, fake_url_percentage]
no_url_percentages = [100 - real_url_percentage, 100 - fake_url_percentage]

fig, ax = plt.subplots(figsize=(10, 6))
width = 0.35
x = np.arange(len(labels))

ax.bar(x, url_percentages, width, label='Contains URLs', color='#FF6B6B')
ax.bar(x, no_url_percentages, width, bottom=url_percentages, label='No URLs', color='#4ECDC4')

ax.set_ylabel('Percentage')
ax.set_title('URL Presence in News Articles')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# Add percentage labels
for i, v in enumerate(url_percentages):
    ax.text(i, v/2, f"{v:.1f}%", ha='center', va='center', color='white', fontweight='bold')
    ax.text(i, v + (no_url_percentages[i]/2), f"{no_url_percentages[i]:.1f}%", ha='center', va='center', color='white', fontweight='bold')

plt.tight_layout()
plt.show()

## Analyzing Social Media Content

Let's check for Twitter handles and hashtags in the datasets.

In [ ]:
# Define functions to check for Twitter handles and hashtags
def contains_twitter_handle(text):
    if text is None:
        return False
    pattern = r'@\w+'
    return bool(re.search(pattern, text))

def contains_hashtag(text):
    if text is None:
        return False
    pattern = r'#\w+'
    return bool(re.search(pattern, text))

# Register the functions as UDFs
contains_handle_udf = udf(contains_twitter_handle, BooleanType())
contains_hashtag_udf = udf(contains_hashtag, BooleanType())

# Check Twitter handle presence
real_with_handle = real_df.withColumn("contains_handle", contains_handle_udf(col("text")))
real_handle_count = real_with_handle.filter(col("contains_handle") == True).count()
real_handle_percentage = (real_handle_count / real_total) * 100

fake_with_handle = fake_df.withColumn("contains_handle", contains_handle_udf(col("text")))
fake_handle_count = fake_with_handle.filter(col("contains_handle") == True).count()
fake_handle_percentage = (fake_handle_count / fake_total) * 100

# Check hashtag presence
real_with_hashtag = real_df.withColumn("contains_hashtag", contains_hashtag_udf(col("text")))
real_hashtag_count = real_with_hashtag.filter(col("contains_hashtag") == True).count()
real_hashtag_percentage = (real_hashtag_count / real_total) * 100

fake_with_hashtag = fake_df.withColumn("contains_hashtag", contains_hashtag_udf(col("text")))
fake_hashtag_count = fake_with_hashtag.filter(col("contains_hashtag") == True).count()
fake_hashtag_percentage = (fake_hashtag_count / fake_total) * 100

# Print results
print(f"Real news articles containing Twitter handles: {real_handle_count} out of {real_total} ({real_handle_percentage:.2f}%)")
print(f"Fake news articles containing Twitter handles: {fake_handle_count} out of {fake_total} ({fake_handle_percentage:.2f}%)")
print(f"\nReal news articles containing hashtags: {real_hashtag_count} out of {real_total} ({real_hashtag_percentage:.2f}%)")
print(f"Fake news articles containing hashtags: {fake_hashtag_count} out of {fake_total} ({fake_hashtag_percentage:.2f}%)")

# Plot the results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Twitter handles plot
handle_percentages = [real_handle_percentage, fake_handle_percentage]
no_handle_percentages = [100 - real_handle_percentage, 100 - fake_handle_percentage]

ax1.bar(x, handle_percentages, width, label='Contains Twitter Handles', color='#FF6B6B')
ax1.bar(x, no_handle_percentages, width, bottom=handle_percentages, label='No Twitter Handles', color='#4ECDC4')
ax1.set_ylabel('Percentage')
ax1.set_title('Twitter Handle Presence in News Articles')
ax1.set_xticks(x)
ax1.set_xticklabels(labels)
ax1.legend()

# Hashtags plot
hashtag_percentages = [real_hashtag_percentage, fake_hashtag_percentage]
no_hashtag_percentages = [100 - real_hashtag_percentage, 100 - fake_hashtag_percentage]

ax2.bar(x, hashtag_percentages, width, label='Contains Hashtags', color='#FF6B6B')
ax2.bar(x, no_hashtag_percentages, width, bottom=hashtag_percentages, label='No Hashtags', color='#4ECDC4')
ax2.set_ylabel('Percentage')
ax2.set_title('Hashtag Presence in News Articles')
ax2.set_xticks(x)
ax2.set_xticklabels(labels)
ax2.legend()

plt.tight_layout()
plt.show()

## Date Format Analysis

Let's check the date formats in both datasets.

In [ ]:
# Create a DateValidator instance
date_validator = DateValidator()

# Check date formats in real news
print("Date format examples in real news:")
display(real_df.select("date").limit(5))

# Check date formats in fake news
print("\nDate format examples in fake news:")
display(fake_df.select("date").limit(5))

# Check for trailing spaces in dates
real_with_trailing = real_df.filter(expr("length(date) != length(trim(date))")).count()
fake_with_trailing = fake_df.filter(expr("length(date) != length(trim(date))")).count()

print(f"\nReal news dates with trailing spaces: {real_with_trailing} out of {real_total} ({(real_with_trailing/real_total)*100:.2f}%)")
print(f"Fake news dates with trailing spaces: {fake_with_trailing} out of {fake_total} ({(fake_with_trailing/fake_total)*100:.2f}%)")

## Dataset-Specific Preprocessing

Based on our analysis, we'll now apply dataset-specific preprocessing to address the identified issues.

In [ ]:
# Create an EnhancedTextPreprocessor instance
preprocessor = EnhancedTextPreprocessor()

# Add label column to both datasets
real_with_label = real_df.withColumn("label", lit(1))
fake_with_label = fake_df.withColumn("label", lit(0))

# Combine datasets
combined_df = real_with_label.union(fake_with_label)

# Apply preprocessing
preprocessed_df = preprocessor.preprocess_spark_df(
    combined_df, 
    text_column="text", 
    title_column="title", 
    combine_title_text=True
)

# Display sample of preprocessed data
print("Sample of preprocessed data:")
display(preprocessed_df.select("title", "text", "processed_text", "label").limit(5))

## Tokenization

Let's tokenize the preprocessed text for further analysis.

In [ ]:
# Tokenize the preprocessed text
tokenized_df = preprocessor.tokenize_spark_df(preprocessed_df, "processed_text")

# Display sample of tokenized data
print("Sample of tokenized data:")
display(tokenized_df.select("processed_text", "tokens", "label").limit(5))

## Extract Quoted Content

Let's extract quoted content from the text for additional features.

In [ ]:
# Extract quoted content
quoted_df = preprocessor.extract_quoted_content_spark_df(preprocessed_df, "text")

# Display sample of data with quoted content
print("Sample of data with extracted quoted content:")
display(quoted_df.select("text", "quoted_content", "label").limit(5))

## Save Preprocessed Data

Finally, let's save the preprocessed data for the feature engineering stage.

In [ ]:
# Save the preprocessed data
preprocessed_df.write.mode("overwrite").parquet("/dbfs/FileStore/fake_news_detection/preprocessed_data")

# Save tokenized data
tokenized_df.write.mode("overwrite").parquet("/dbfs/FileStore/fake_news_detection/tokenized_data")

# Save quoted content data
quoted_df.write.mode("overwrite").parquet("/dbfs/FileStore/fake_news_detection/quoted_content_data")

print("Preprocessed data saved successfully.")

## Summary of Preprocessing Steps

In this notebook, we have:

1. **Loaded and analyzed** the raw data from Hive tables
2. **Identified dataset-specific preprocessing needs**:
   - Demonstrated why the 'subject' column must be dropped (perfect data leakage)
   - Confirmed the need for URL removal, especially in fake news articles
   - Identified social media content (Twitter handles, hashtags) that requires special handling
   - Found trailing spaces in dates that need trimming
3. **Applied dataset-specific preprocessing**:
   - Dropped the 'subject' column to prevent data leakage
   - Enhanced URL removal with specific patterns for Twitter URLs
   - Handled social media artifacts (Twitter handles, hashtags)
   - Normalized text and trimmed whitespace
   - Combined title and text for more comprehensive analysis
4. **Tokenized text** for feature engineering
5. **Extracted quoted content** for additional features
6. **Saved preprocessed data** for the next stage of the pipeline

These preprocessing steps ensure that our model will learn from the actual content of the news articles rather than artificial distinctions or irrelevant features.